In [26]:
from shapely.geometry import MultiPoint, LineString, MultiLineString, Point
from shapely.ops import nearest_points, linemerge
import numpy as np
# from libpysal.cg.voronoi import voronoi, voronoi_frames
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.ops import unary_union, polygonize, cascaded_union, split, polygonize_full
from shapely import affinity
from datetime import datetime
from os.path import exists
import pygeos as pg
import os

In [27]:
df_qv = gpd.read_file('gis/SIRGAS_GPKG_quadraviariaed_2017.gpkg')
df_l = gpd.read_file(f'zip://gis/SIRGAS_SHP_logradouronbl.zip!SIRGAS_SHP_logradouronbl/SIRGAS_SHP_logradouronbl.shp')

In [28]:
df_dist = gpd.read_file(f'zip://gis/SIRGAS_SHP_distrito.zip!SIRGAS_SHP_distrito/SIRGAS_SHP_distrito_polygon.shp')
df_represas = gpd.read_file(f'zip://gis/SIRGAS_SHP_represanivelmax.zip!SIRGAS_SHP_represanivelmax').set_crs(epsg=31983)
df_massa_dagua = gpd.read_file(f'zip://gis/SIRGAS_SHP_hidrogramassadagua.zip!SIRGAS_SHP_hidromassadagua/SIRGAS_SHP_hidromassadagua_polygon.shp').set_crs(epsg=31983)

In [29]:
df_cruzamentos = gpd.read_file(f'resultado/pre-processamentos/00_cruzamentos_cidade_sao_paulo.gpkg')

In [176]:
for id, dist in df_dist.iterrows():

    print(f'Processando meio-fio de {dist.ds_nome}')
    if exists(f'resultado/pre-processamentos/{dist.ds_codigo}_meio-fio_{dist.ds_nome}.gpkg'):
        continue

    qv = df_qv.loc[df_qv.qe_tipo == 'Quadra'].clip(dist.geometry)
    qv = gpd.GeoDataFrame(geometry=qv.buffer(0.001))
    l = df_l.clip(dist.geometry.buffer(20))
    # l = gpd.GeoDataFrame(geometry=l)

    cruzamentos = df_cruzamentos.clip(dist.geometry)
    cruzamentos_buffer = gpd.GeoDataFrame(geometry=cruzamentos.buffer(35)).set_crs(epsg=31983)
    
    esquinas = qv.overlay(cruzamentos_buffer, how='intersection').explode(index_parts=False).reset_index()
    # esquinas = gpd.GeoDataFrame(geometry=[esquinas]).explode(index_parts=True).reset_index()

    cruzamento_esquina = gpd.sjoin(esquinas, cruzamentos_buffer, how='left', predicate='intersects').reset_index()
    # cruzamento_esquina = cruzamento_esquina[~cruzamento_esquina.index_right.isna()]

    linhas_corte = MultiLineString([LineString(nearest_points(r.geometry, cruzamentos.loc[r.index_right].geometry)) 
                            for i, r in cruzamento_esquina.iterrows()])

    linhas_unidas = qv.boundary.unary_union.union(linhas_corte)
    geometrias = [i for i in linemerge(linhas_unidas)]

    vias_union = dist.geometry.difference(qv.buffer(0.30).unary_union)
    qv_union = gpd.GeoDataFrame(geometry=qv.buffer(-0.30)).unary_union

    meios_fios = gpd.GeoDataFrame(geometry=geometrias, crs=31983)

    meio_fio_ponto_medio = gpd.GeoDataFrame(meios_fios.copy(), geometry=[l.interpolate(0.5, normalized=True) for l in meios_fios.geometry]).sjoin_nearest(l, how='left').drop_duplicates()

    meios_fios.loc[meio_fio_ponto_medio['lg_codlog'].index, 'lg_codlog'] = meio_fio_ponto_medio['lg_codlog']
    meios_fios.loc[meio_fio_ponto_medio['lg_seg_id'].index, 'lg_seg_id'] = meio_fio_ponto_medio['lg_seg_id']
    meios_fios.loc[meio_fio_ponto_medio.index, 'lado_par'] = [r.geometry.intersects(l.loc[r.index_right].geometry.buffer(35, single_sided=True)) for i, r in meio_fio_ponto_medio.iterrows()]

    meios_fios = meios_fios.dissolve(by=['lg_codlog', 'lg_seg_id', 'lado_par']).reset_index()

    meios_fios = meios_fios.explode(index_parts=False).reset_index()
    # meios_fios = meios_fios[~meios_fios.intersects(vias_union)]
    # meios_fios = meios_fios[~meios_fios.intersects(qv_union)]

    meios_fios = meios_fios.dissolve(by=['lg_codlog', 'lg_seg_id', 'lado_par']).reset_index()

    meios_fios.to_file(f'resultado/pre-processamentos/{dist.ds_codigo}_meio-fio_{dist.ds_nome}.gpkg', 
                        driver='GPKG', crs=31983)

    break

Processando meio-fio de MOEMA


/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [173]:
meio_fio_ponto_medio

,geometry,index_right,lg_codlog,lg_tipo,lg_titulo,lg_prep,lg_nome,lg_or_deno,lg_ind_ofi,lg_ini_par,lg_fim_par,lg_ini_imp,lg_fim_imp,lg_ordem,lg_id,lg_seg_id,lg_or_geom
0,POINT (329022.908 7388134.955),202620,108340,R,DR,None,JOSE CANDIDO DE SOUZA,None,OFICIAL,0.0,118.0,1.0,119.0,1.0,40564.0,176995,None
1,POINT (328979.059 7388174.664),202620,108340,R,DR,None,JOSE CANDIDO DE SOUZA,None,OFICIAL,0.0,118.0,1.0,119.0,1.0,40564.0,176995,None
2,POINT (329000.118 7388078.925),117554,191000,AV,None,DOS,BANDEIRANTES,None,OFICIAL,NaN,NaN,2179.0,2259.0,167.0,48928.0,177544,None
3,POINT (328993.949 7388208.532),19178,009997,AV,STO,None,AMARO,None,OFICIAL,4768.0,4838.0,NaN,NaN,133.0,28568.0,175654,None
4,POINT (329025.716 7388141.804),202620,108340,R,DR,None,JOSE CANDIDO DE SOUZA,None,OFICIAL,0.0,118.0,1.0,119.0,1.0,40564.0,176995,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9247,POINT (331681.089 7390180.972),202789,349968,AC,None,None,A AV IBIRAPUERA,None,OFICIAL,0.0,146.0,NaN,NaN,3.0,64763.0,176620,None
9248,POINT (331684.843 7390173.967),113176,349968,AC,None,None,A AV IBIRAPUERA,None,OFICIAL,0.0,142.0,1.0,143.0,1.0,64763.0,177626,None
9249,POINT (331696.362 7390188.246),163118,089443,AV,None,None,IBIRAPUERA,None,OFICIAL,0.0,220.0,1.0,221.0,1.0,38640.0,176558,None
9250,POINT (331797.154 7390303.774),163118,089443,AV,None,None,IBIRAPUERA,None,OFICIAL,0.0,220.0,1.0,221.0,1.0,38640.0,176558,None


In [172]:
meios_fios

,geometry,lg_codlog,lg_seg_id
0,"LINESTRING (328981.559 7388165.823, 328982.010...",108340,176995.0
1,"LINESTRING (328981.559 7388165.823, 328978.473...",108340,176995.0
2,"LINESTRING (329061.900 7388100.243, 329061.858...",191000,177544.0
3,"LINESTRING (328985.370 7388178.125, 328985.214...",009997,175654.0
4,"LINESTRING (329068.134 7388107.690, 329067.865...",108340,176995.0
...,...,...,...
9247,"LINESTRING (331677.242 7390184.261, 331684.936...",349968,176620.0
9248,"LINESTRING (331684.749 7390170.251, 331684.936...",349968,177626.0
9249,"LINESTRING (331707.788 7390198.809, 331684.936...",089443,176558.0
9250,"LINESTRING (331707.788 7390198.809, 331707.788...",089443,176558.0


In [159]:
meio_fio_ponto_medio

,geometry,index_right,lg_codlog,lg_tipo,lg_titulo,lg_prep,lg_nome,lg_or_deno,lg_ind_ofi,lg_ini_par,lg_fim_par,lg_ini_imp,lg_fim_imp,lg_ordem,lg_id,lg_seg_id,lg_or_geom
0,POINT (329022.908 7388134.955),202620,108340,R,DR,None,JOSE CANDIDO DE SOUZA,None,OFICIAL,0.0,118.0,1.0,119.0,1.0,40564.0,176995,None
1,POINT (328979.059 7388174.664),202620,108340,R,DR,None,JOSE CANDIDO DE SOUZA,None,OFICIAL,0.0,118.0,1.0,119.0,1.0,40564.0,176995,None
2,POINT (329000.118 7388078.925),117554,191000,AV,None,DOS,BANDEIRANTES,None,OFICIAL,NaN,NaN,2179.0,2259.0,167.0,48928.0,177544,None
3,POINT (328993.949 7388208.532),19178,009997,AV,STO,None,AMARO,None,OFICIAL,4768.0,4838.0,NaN,NaN,133.0,28568.0,175654,None
4,POINT (329025.716 7388141.804),202620,108340,R,DR,None,JOSE CANDIDO DE SOUZA,None,OFICIAL,0.0,118.0,1.0,119.0,1.0,40564.0,176995,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9247,POINT (331681.089 7390180.972),202789,349968,AC,None,None,A AV IBIRAPUERA,None,OFICIAL,0.0,146.0,NaN,NaN,3.0,64763.0,176620,None
9248,POINT (331684.843 7390173.967),113176,349968,AC,None,None,A AV IBIRAPUERA,None,OFICIAL,0.0,142.0,1.0,143.0,1.0,64763.0,177626,None
9249,POINT (331696.362 7390188.246),163118,089443,AV,None,None,IBIRAPUERA,None,OFICIAL,0.0,220.0,1.0,221.0,1.0,38640.0,176558,None
9250,POINT (331797.154 7390303.774),163118,089443,AV,None,None,IBIRAPUERA,None,OFICIAL,0.0,220.0,1.0,221.0,1.0,38640.0,176558,None


In [127]:
meio_fio_ponto_medio['lg_codlog']

0       108340
2       191000
3       009997
4       108340
5       093289
         ...  
9238    160377
9244    106399
9246    349950
9250    089443
9251    186538
Name: lg_codlog, Length: 3820, dtype: object

In [123]:
meios_fios.loc[meio_fio_ponto_medio['lg_seg_id'].index, 'lg_seg_id'] = meio_fio_ponto_medio['lg_seg_id']

In [110]:
meios_fios

,geometry
0,"LINESTRING (328981.559 7388165.823, 328982.010..."
2,"LINESTRING (329061.900 7388100.243, 329061.858..."
3,"LINESTRING (328985.370 7388178.125, 328985.214..."
4,"LINESTRING (329068.134 7388107.690, 329067.865..."
5,"LINESTRING (329005.663 7388237.854, 329006.701..."
...,...
9238,"LINESTRING (331658.394 7390847.262, 331670.808..."
9244,"LINESTRING (331669.933 7390867.273, 331669.933..."
9246,"LINESTRING (331676.208 7390070.100, 331676.208..."
9250,"LINESTRING (331707.788 7390198.809, 331707.788..."
